In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import proplot as pplt
import seaborn as sns
import pandas as pd
import torch
import gymnasium as gym
import gym_bart
import itertools
from umap import UMAP
from plotting_utils import (
    set_rc, 
    add_abc_to_subaxes, 
    colors, 
    rgb_colors
)
from model_evaluation import (
    forced_action_evaluate, 
    meta_bart_callback,
    meta_bart_multi_callback,
    reshape_parallel_evalu_res,
    forced_action_evaluate_multi,
)
from bart_behavior_analysis import (
    plot_1color5fsize,
    plot_1colornfsize
)
from bart_single_node_responses import *
from read_experiments import average_runs, load_exp_df
import re

from functools import partial
import sys
sys.path.append('../')
from ppo.envs import make_vec_env
import pickle
from tqdm import tqdm
from collections import defaultdict
from pathlib import Path

from bart_representation_analysis import *
from bart_compress_visualize_decode import *
from sklearn.decomposition import PCA
from scipy.stats import ttest_ind

set_rc()

In [ ]:
res = pickle.load(open('data/meta_representation_results', 'rb')) # arrays
ares = pickle.load(open('data/meta_rnn_gradients', 'rb')) # dicts
forced_ares = pickle.load(open('data/meta_rnn_forced_gradients', 'rb')) # dicts
node_ares = pickle.load(open('data/meta_single_node_characteristics', 'rb')) # arrays

In [3]:
print('res (general)')
print(np.array(list(res.keys())))
print('ares (activations)')
print(np.array(list(ares.keys())))
print('forced_ares (meta clustering)')
print(np.array(list(forced_ares.keys())))
print('node_ares (single node character)')
print(np.array(list(node_ares.keys())))

res (general)
['last_sizes' 'unpopped_sizes' 'pop_rate' 'rewards' 'values'
 'action_probs' 'all_lens' 'all_num_balloons' 'balloon_steps'
 'button_presses' 'dec_flow_scores' 'iterators_idxs' 'sizes' 'ramp_f1s'
 'ramp_indiv_contribs' 'confidence_scores' 'unconfidence_scores'
 'unconfident_points' 'step_count' 'all_decision_nodes'
 'cluster_regressor_coefs' 'cluster_regressor_scores' 'cluster_ks'
 'pca_regressor_coefs' 'pca_regressor_scores']
ares (activations)
['cluster_activations' 'cluster_labels' 'cluster_orientations'
 'rnn_hx_influences' 'val_grads' 'action_grads' 'all_rnn_hxs'
 'all_ep_lens' 'all_obs']
forced_ares (meta clustering)
['cluster_activations' 'rnn_hx_influences' 'val_grads' 'action_grads'
 'all_rnn_hxs' 'fixprev_res' 'give_res' 'res']
node_ares (single node character)
['response_types' 'turning_points' 'lr_sens_bias' 'size_sensitivities'
 'pop_sensitivities' 'reversals']


So far, we've used the following primarily

**res**
- 